In [2]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [3]:
df = pd.read_csv("data/HR-Employee-Attrition.csv")
df.pop("Attrition")

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Age': 32,
 'BusinessTravel': 'Travel_Rarely',
 'DailyRate': 334,
 'Department': 'Research & Development',
 'DistanceFromHome': 5,
 'Education': 2,
 'EducationField': 'Life Sciences',
 'EmployeeCount': 1,
 'EnvironmentSatisfaction': 1,
 'Gender': 'Male',
 'HourlyRate': 80,
 'JobInvolvement': 4,
 'JobLevel': 1,
 'JobRole': 'Research Scientist',
 'JobSatisfaction': 2,
 'MaritalStatus': 'Divorced',
 'MonthlyIncome': 3298,
 'MonthlyRate': 15053,
 'NumCompaniesWorked': 0,
 'Over18': 'Y',
 'OverTime': 'Yes',
 'PercentSalaryHike': 12,
 'PerformanceRating': 3,
 'RelationshipSatisfaction': 4,
 'StandardHours': 80,
 'StockOptionLevel': 2,
 'TotalWorkingYears': 7,
 'TrainingTimesLastYear': 5,
 'WorkLifeBalance': 2,
 'YearsAtCompany': 6,
 'YearsInCurrentRole': 2,
 'YearsSinceLastPromotion': 0,
 'YearsWithCurrManager': 5}

In [4]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [5]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [6]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)
    
    endpoint = "https://employee-pred-production.up.railway.app/v1/models/employee-attrition-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "No Attrition"
    else:
        return "Yes Attrition"

In [7]:
make_predictions(inputs)

'No Attrition'